In [1]:
from __future__ import annotations

import json

In [2]:
from py_research.db import (
    DataSource,
    RecMap,
    RelMap,
    SubMap,
)

In [3]:
from test_db_schema import Search, Project, Task, User, Membership, Organization

In [4]:
data_source = DataSource(
    rec=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": RelMap(
            rel=Search.results,
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": RelMap(
                    rel=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": RelMap(
                            rel=Task.assignees,
                            push=User.name,
                            match=User.name,
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": RelMap(
                    rel=Project.members,
                    push={User.name, User.age},
                    link=RecMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                    },
                ),
            },
        ),
    },
)

In [5]:
with open("./nested_data.json") as f:
    data = json.load(f)

In [ ]:
rec = data_source.load(data)
db = data_source.cache